In [27]:
from google.cloud import vision
from pdf2image import convert_from_path
from PIL import Image, ImageDraw, ImageFont
import io
import fitz  # PyMuPDF
import os
from xml.sax.saxutils import escape


In [28]:
client = vision.ImageAnnotatorClient()

I0000 00:00:1731625236.360544 4888277 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [29]:
def get_image_metadata_from_pdf(pdf_path):
    """
    Extracts metadata of images embedded in each page of a scanned PDF,
    including page dimensions in points and inches, and calculates DPI if missing.
    """
    doc = fitz.open(pdf_path)
    image_metadata = []
    
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        
        # Get page dimensions in points and inches
        page_width_pts, page_height_pts = page.rect.width, page.rect.height
        page_width_in = page_width_pts / 72  # Convert points to inches
        page_height_in = page_height_pts / 72
        
        # Extract the list of images on this page
        images = page.get_images(full=True)  # 'full=True' gets all images on page
        
        for img_index, img in enumerate(images):
            xref = img[0]  # Image reference number
            
            try:
                # Attempt to extract the image and retrieve its metadata
                base_image = doc.extract_image(xref)
                
                # Retrieve metadata about the image
                width_px, height_px = base_image["width"], base_image["height"]
                dpi_x, dpi_y = base_image.get("dpi", (None, None))
                image_metadata.append({
                            "index": img_index,
                            "width": width_px,
                            "height": height_px,
                            "dpi_x": dpi_x,
                            "dpi_y": dpi_y
                        })
            except Exception as e:
                print(f"Error extracting image from {pdf_path}, page {page_num}, at index {img_index}: {e}")
                continue  # Skip to the next image if extraction fails
            
            # Calculate DPI if not provided
            if dpi_x is None or dpi_y is None:
                dpi_x = width_px / page_width_in
                dpi_y = height_px / page_height_in
            
            # Append metadata, including page dimensions and calculated DPI
            image_metadata.append({
                "page": page_num + 1,
                "image_index": img_index + 1,
                "width_px": width_px,
                "height_px": height_px,
                "format": base_image["ext"],
                "dpi_x": dpi_x,
                "dpi_y": dpi_y,
                "page_width_pts": page_width_pts,
                "page_height_pts": page_height_pts,
                "page_width_in": page_width_in,
                "page_height_in": page_height_in,
            })
    
    doc.close()
    return image_metadata

In [31]:
def build_processed_pdf(directory, filename):
    pil_images = []
    text_pages = []

    file_path = os.path.join(directory, filename)
    print(f"beginning to process: {file_path}")

    # Load PDF and extract image bytes from a given page
    doc = fitz.open(file_path)

    for page_num in range(doc.page_count):
        print(f"processing page: {page_num}...")
        
        page = doc.load_page(page_num)  # Load the first page
        images = page.get_images(full=True)
        xref = images[0][0]  # Get the first image reference
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]  # Get the raw JPEG byte content

        # Convert to PIL Image for further processing if needed
        p_image = Image.open(io.BytesIO(image_bytes))
        pil_images.append(p_image)

        # Annotate with vision
        image = vision.Image(content=image_bytes)
        response = client.text_detection(image=image)
        text_annotations = response.text_annotations

        # Build labeled image
        font_size = 15  
        font_path="/System/Library/Fonts/Supplemental/Arial.ttf"
        font = ImageFont.truetype(font_path, font_size)

        draw = ImageDraw.Draw(p_image)
        for i, text_obj in enumerate(text_annotations):
            # Get the bounding box vertices
            vertices = [(vertex.x, vertex.y) for vertex in text_obj.bounding_poly.vertices]

            # Draw a polygon based on the bounding box
            if len(vertices) == 4:
                draw.polygon(vertices, outline="red", width=2)
                if i > 0: # first bounding box is whole text region
                    draw.text(vertices[0], text_obj.description, fill="red", font=font)

        full_page_text = text_annotations[0].description  # always the first text_annotation object
        text_pages.append(full_page_text)

    return text_pages, pil_images

In [32]:
import os
import json

def save_annotated_text_and_image_data(root_dir, data_location, text_data, image_data):
    """
    Creates a directory structure to save images, text data, and an HTML gallery for viewing.
    
    Parameters:
        root_dir (str): The root directory to store data.
        data_location (str): Directory location under root_dir for storing files.
        text_data (list of str): List of extracted text for each page.
        image_data (list of tuples): List of tuples, where each tuple contains the image filename (str) 
                                     and the image data (as bytes or PIL image).
                                     
    """
    # Create directory paths
    base_dir = os.path.join(root_dir, data_location)
    os.makedirs(base_dir, exist_ok=True)

    images_dir = os.path.join(base_dir, "images")
    os.makedirs(images_dir, exist_ok=True)
    
    # File paths for JSONL and HTML files
    jsonl_file_path = os.path.join(base_dir, "extracted_text.jsonl")
    html_file_path = os.path.join(base_dir, "index.html")
    
    # Step 1: Save text data to JSONL
    with open(jsonl_file_path, "w", encoding="utf-8") as jsonl_file:
        for i, text in enumerate(text_data):
            entry = {
                "image": f"images/page{i+1}.jpg",  # Image filename
                "text": text
            }
            jsonl_file.write(json.dumps(entry) + "\n")
    
    # Step 2: Save images to the images directory
    for i, img in enumerate(image_data):
        image_path = os.path.join(images_dir, f"page{i+1}.jpg")
        img.save(image_path, format="JPEG")
    
    # Step 3: Generate HTML file for viewing
    with open(html_file_path, "w", encoding="utf-8") as html_file:
        html_file.write("<html><head><title>Annotated Image Gallery</title></head><body>\n")
        html_file.write("<h1>Annotated Image Gallery</h1>\n")
        
        for i, text in enumerate(text_data):
            img_filename = f"images/page{i+1}.jpg"
            html_file.write("<div class='image-entry'>\n")
            html_file.write(f"  <img src='{img_filename}' alt='Page {i+1}' style='width:100%; max-width:600px;'>\n")
            html_file.write(f"  <textarea readonly style='width:100%; height:150px;'>{text}</textarea>\n")
            html_file.write("</div><br>\n")
        
        html_file.write("</body></html>")

    print(f"Data successfully saved in {base_dir}")

# Example usage:
# save_annotated_text_and_image_data(root_dir="my_data", data_location="ocr_project", text_data=["text1", "text2"], image_data=[("img1.jpg", img_bytes), ("img2.jpg", img_bytes)])

In [34]:
import os

def save_xml_text_file(root_dir, title_file, text_data):
    """
    Generates and saves an XML file with a title and pages, where each page contains text data.
    
    Parameters:
        root_dir (str): The root directory where the XML file will be saved.
        title_file (str): The title to be used in the XML file and filename.
        text_data (list of str): List of extracted text for each page.
    """
    # Define the XML file path
    xml_file_path = os.path.join(root_dir, title_file, f"full_OCR_text_{title_file}.xml")
    os.makedirs(os.path.dirname(xml_file_path), exist_ok=True)
    
    # Start writing the XML file
    with open(xml_file_path, "w", encoding="utf-8") as xml_file:
        # Write XML declaration and root element with title
        xml_file.write("<?xml version='1.0' encoding='UTF-8'?>\n")
        xml_file.write(f"<document>\n  <title>{escape(title_file)}</title>\n")
        
        # Add each page with its content, escaping the text for XML safety
        for i, text in enumerate(text_data):
            # Escape text content to handle special characters
            escaped_text = escape(text)
            # Write each page as <page page="1">...</page>
            xml_file.write(f"  <page page='{i+1}'>\n")
            xml_file.write(f"    {escaped_text}\n")
            xml_file.write("  </page>\n")
        
        # Close the root element
        xml_file.write("</document>")

    print(f"XML file successfully saved at {xml_file_path}")

In [35]:
def full_pdf_process(pdf_dir, pdf_filename, output_dir):
    current_title, ext = os.path.splitext(pdf_filename)
    assert ext == ".pdf", f"Expected .pdf, got {ext}"
    text_pages, marked_images = build_processed_pdf(pdf_dir, pdf_filename)
    save_annotated_text_and_image_data(output_dir, current_title, text_pages, marked_images)
    save_xml_text_file(output_dir, current_title, text_pages)
    return text_pages, marked_images


In [21]:
# directory to read pdfs from
pdf_dir = "../PDF/Phat_Giao_journals"

# directory to save output
output_dir = "../processed_journal_data"

In [22]:
text_pages, marked_images = full_pdf_process(pdf_dir, "phat-giao-viet-nam-1956-25-26.pdf", output_dir)

beginning to process: ../PDF/Phat_Giao_journals/phat-giao-viet-nam-1956-25-26.pdf
processing page: 0...
processing page: 1...
processing page: 2...
processing page: 3...
processing page: 4...
processing page: 5...
processing page: 6...
processing page: 7...
processing page: 8...
processing page: 9...
processing page: 10...
processing page: 11...
processing page: 12...
processing page: 13...
processing page: 14...
processing page: 15...
processing page: 16...
processing page: 17...
processing page: 18...
processing page: 19...
processing page: 20...
processing page: 21...
processing page: 22...
processing page: 23...
processing page: 24...
processing page: 25...
processing page: 26...
processing page: 27...
Data successfully saved in ../processed_journal_data/phat-giao-viet-nam-1956-25-26
XML file successfully saved at ../processed_journal_data/phat-giao-viet-nam-1956-25-26/full_OCR_text_phat-giao-viet-nam-1956-25-26.xml


In [38]:
all_files =  os.listdir(pdf_dir)

In [42]:
all_files

['phat-giao-viet-nam-1956-15.pdf',
 'phat-giao-viet-nam-1956-01.pdf',
 'phat-giao-viet-nam-1956-28.pdf',
 'phat-giao-viet-nam-1956-14.pdf',
 'phat-giao-viet-nam-1956-02.pdf',
 'phat-giao-viet-nam-1956-16.pdf',
 'phat-giao-viet-nam-1956-20-21.pdf',
 'phat-giao-viet-nam-1956-03.pdf',
 'phat-giao-viet-nam-1956-07.pdf',
 'phat-giao-viet-nam-1956-13.pdf',
 'phat-giao-viet-nam-1956-12.pdf',
 'phat-giao-viet-nam-1956-04.pdf',
 'phat-giao-viet-nam-1956-11.pdf',
 'phat-giao-viet-nam-1956-05-06.pdf',
 'phat-giao-viet-nam-1956-09-10.pdf',
 'phat-giao-viet-nam-1956-17-18.pdf',
 'phat-giao-viet-nam-1956-25-26.pdf',
 'phat-giao-viet-nam-1956-23.pdf',
 'phat-giao-viet-nam-1956-22.pdf',
 'phat-giao-viet-nam-1956-27.pdf',
 'phat-giao-viet-nam-1956-19.pdf',
 'phat-giao-viet-nam-1956-24.pdf']

In [43]:
px_list = [all_files[2]] + all_files[6:16] + all_files[17:21]

In [44]:
px_list

['phat-giao-viet-nam-1956-28.pdf',
 'phat-giao-viet-nam-1956-20-21.pdf',
 'phat-giao-viet-nam-1956-03.pdf',
 'phat-giao-viet-nam-1956-07.pdf',
 'phat-giao-viet-nam-1956-13.pdf',
 'phat-giao-viet-nam-1956-12.pdf',
 'phat-giao-viet-nam-1956-04.pdf',
 'phat-giao-viet-nam-1956-11.pdf',
 'phat-giao-viet-nam-1956-05-06.pdf',
 'phat-giao-viet-nam-1956-09-10.pdf',
 'phat-giao-viet-nam-1956-17-18.pdf',
 'phat-giao-viet-nam-1956-23.pdf',
 'phat-giao-viet-nam-1956-22.pdf',
 'phat-giao-viet-nam-1956-27.pdf',
 'phat-giao-viet-nam-1956-19.pdf']

In [45]:
for filename in px_list:
        #print(filename)  # Do something with each file path
        full_pdf_process(pdf_dir, filename, output_dir)

beginning to process: ../PDF/Phat_Giao_journals/phat-giao-viet-nam-1956-28.pdf
processing page: 0...
processing page: 1...
processing page: 2...
processing page: 3...
processing page: 4...
processing page: 5...
processing page: 6...
processing page: 7...
processing page: 8...
processing page: 9...
processing page: 10...
processing page: 11...
processing page: 12...
processing page: 13...
processing page: 14...
processing page: 15...
processing page: 16...
processing page: 17...
processing page: 18...
processing page: 19...
processing page: 20...
processing page: 21...
processing page: 22...
processing page: 23...
processing page: 24...
processing page: 25...
processing page: 26...
processing page: 27...
processing page: 28...
processing page: 29...
processing page: 30...
processing page: 31...
processing page: 32...
processing page: 33...
processing page: 34...
processing page: 35...
processing page: 36...
processing page: 37...
processing page: 38...
processing page: 39...
processing 